# File formats

In principle, Niimpy can deal with any files of any format - you only need to convert them to a DataFrame.  Still, it is very useful to have some common formats, so we present two standard formats with default readers:

* **CSV files** are very standard and normal to create and understand, but in order to deal with them everything must be loaded into memory.
* **sqlite3 databases**, which requires sqlite3 to read, but provides more power for filtering and automatic processing without reading everything into memory.
* **Google TakeOut** provides a large selection of data in different formats. We provide readers most commonly used data types.
* **MHealth** is a common format for health data.

## DataFrame format (in-memory)

In-memory, data is stored in a [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).  This is basically a normal dataframe.  There are some standardized columns (see the [schema](schema.html)) and the index is a DatetimeIndex.

## CSV files

CSV files should have a header that lists the column names and generally be readable by `pandas.read_csv`.

Reading these can be done with `niimpy.read_csv`:

In [1]:
import os
import niimpy 
import niimpy.config as config

# Read the battery data
df= niimpy.read_csv(config.MULTIUSER_AWARE_BATTERY_PATH, tz='Europe/Helsinki')

## sqlite3 databases

For the purposes of niimpy, sqlite3 databases can generally be seen as supercharged CSV files.

A single database file could contain multiple datasets within it, thus when reading them a **table name** must be specified.

One reads the entire database into memory using `sqlite.read_sqlite`:

In [2]:
# Read the sqlite3 data
df= niimpy.read_sqlite(config.SQLITE_SINGLEUSER_PATH, table="AwareScreen", tz='Europe/Helsinki')

You can list the tables within a database using `niimpy.reading.read.read_sqlite_tables`:

In [3]:
niimpy.reading.read.read_sqlite_tables(config.SQLITE_SINGLEUSER_PATH)

{'AwareScreen'}

sqlite3 files are highly recommended as a data storage format, since many common exploration options can be done within the database itself without reading the whole data into memory or writing an iterator.  However, the interface is more difficult to use.  Niimpy (before 2021-07) used this as its primary interface, but since then this interface has been de-emphasized.  You can read more in [the database section](database.html), but this is only recommended if you need efficiency when using massive amounts of data.

## Google TakeOut

Google takeout contains a many different types of data and new types are added as Google creates services or changes data storage methods. Readers are currently available for location data and activity data from the fit app. For other data types, the user needs to manually convert them into a Niimpy compatible Pandas DataFrame.

In [7]:
import niimpy
import niimpy.config as config
import niimpy.preprocessing.location as nilo

data = niimpy.reading.google_takeout.location_history(config.GOOGLE_TAKEOUT_PATH)
data = nilo.filter_location(
    data,
    latitude_column = "latitude",
    longitude_column = "longitude",
    remove_disabled=False, remove_network=False, remove_zeros=True
)
data.head()


,accuracy,source,device,placeid,formfactor,latitude,longitude,inferred_latitude,inferred_longitude,activity_type,activity_inference_confidence,user
timestamp,,,,,,,,,,,,
2016-08-12 19:29:43.821000+00:00,25,WIFI,-577680260,NaN,NaN,35.997488,-78.922194,NaN,NaN,NaN,NaN,9e9cff5a-93a0-11ee-addd-b0dcef010c43
2016-08-12 19:30:49.531000+00:00,21,WIFI,-577680260,NaN,NaN,35.997559,-78.922504,NaN,NaN,STILL,62.0,9e9cff5a-93a0-11ee-addd-b0dcef010c43
2016-08-12 19:31:49.531000+00:00,21,WIFI,-577680260,ChIJS_5Nmuz1jUYRGYf3QiiZco4,PHONE,35.997559,-78.922504,60.187135,24.824478,STILL,62.0,9e9cff5a-93a0-11ee-addd-b0dcef010c43
2016-08-12 21:15:55.295000+00:00,1500,CELL,-577680260,NaN,NaN,36.000870,-78.923343,NaN,NaN,ON_FOOT,54.0,9e9cff5a-93a0-11ee-addd-b0dcef010c43
2016-08-12 21:16:33+00:00,8,GPS,-577680260,NaN,NaN,35.997250,-78.923989,NaN,NaN,NaN,NaN,9e9cff5a-93a0-11ee-addd-b0dcef010c43


In [9]:
nilo.location_distance_features(data, {
    "latitude_column": "latitude",
    "longitude_column": "longitude",
})

,user,device,dist_total,n_bins,speed_average,speed_variance,speed_max,variance,log_variance
timestamp,,,,,,,,,
2016-08-31 00:00:00+00:00,9e9cff5a-93a0-11ee-addd-b0dcef010c43,-577680260,822.227218,6.0,2.013277,15.403444,10.764752,0.000002,-13.007195


Each subject Downloads their Google TakeOut data as a separate zip file. The Zipfile package, which is included in the Python standard, is convenient for reading the data files contained in the zip file. For example, one could read the location data with the following code:

In [5]:
from zipfile import ZipFile
import json
import pandas as pd
from niimpy import config

zip_file = ZipFile(config.GOOGLE_TAKEOUT_PATH)
json_data  = zip_file.read("Takeout/Location History/Records.json")
json_data = json.loads(json_data)
data = pd.json_normalize(json_data["locations"])
data = pd.DataFrame(data)
data.head()

,latitudeE7,longitudeE7,accuracy,source,deviceTag,timestamp,activity,locationMetadata,placeId,formFactor,inferredLocation,activeWifiScan.accessPoints
0,359974880,-789221943,25,WIFI,-577680260,2016-08-12T19:29:43.821Z,NaN,NaN,NaN,NaN,NaN,NaN
1,359975588,-789225036,21,WIFI,-577680260,2016-08-12T19:30:49.531Z,"[{'activity': [{'type': 'STILL', 'confidence':...",NaN,NaN,NaN,NaN,NaN
2,359975588,-789225036,21,WIFI,-577680260,2016-08-12T19:31:49.531Z,"[{'activity': [{'type': 'STILL', 'confidence':...",[{'wifiScan': {'accessPoints': [{'mac': '12410...,ChIJS_5Nmuz1jUYRGYf3QiiZco4,PHONE,"[{'timestamp': '2023-11-21T10:40:35.320Z', 'la...","[{'mac': '124103876652832', 'strength': -63, '..."
3,360008703,-789233433,1500,CELL,-577680260,2016-08-12T21:15:55.295Z,"[{'activity': [{'type': 'ON_FOOT', 'confidence...",NaN,NaN,NaN,NaN,NaN
4,359972502,-789239894,8,GPS,-577680260,2016-08-12T21:16:33Z,NaN,NaN,NaN,NaN,NaN,NaN


Location data is stored in the json format. Other types of data are stored in various formats and with different files structures. The user must find how each type of data they need is stored and how it can be read in Python.

## Other formats

You can add readers for any types of formats which you can convert into a Pandas dataframe (so basically anything).  For examples of readers, see `niimpy/reading/read.py`.  Apply the function `niimpy.preprocessing.util.df_normalize` in order to apply some standardizations to get the standard Niimpy format.